In [1]:
import sys
sys.path.append('..')
from bicm.graph_classes import *
from bicm.network_functions import *
# import bicm
# from bicm import *
import numpy as np

This test uses a plant-pollinator network from Petanidou, T. (1991). Pollination ecology in a phryganic ecosystem.
This script is run on an ordinary laptop.

In [2]:
biad_mat_names = np.loadtxt('test_dataset.csv', delimiter=',', dtype=str)
plants = biad_mat_names[1:,0]
pollinators = biad_mat_names[0, 1:]
biad_mat = biad_mat_names[1:, 1:].astype(np.ubyte)
plants_dict = dict(enumerate(np.unique(plants)))
plants_inv_dict = {v:k for k,v in plants_dict.items()}
pollinators_dict = dict(enumerate(np.unique(pollinators)))
pollinators_inv_dict = {v:k for k,v in pollinators_dict.items()}

You can manipulate a data structure into another with the functions of the package. It supports biadjacency matrices (numpy arrays or scipy sparse matrices), edgelists, adjacency lists (dictionaries) or just the degree sequences.

Let's create an edgelist with names.

In [3]:
# These data type converters return additional objects, keeping track of the node labeling and the node degrees.
# The desired data structure is always at index 0. Check the documentation for more info.
edgelist = edgelist_from_biadjacency(biad_mat)[0]
edgelist_names = [(plants_dict[edge[0]], pollinators_dict[edge[1]]) for edge in edgelist]
print(edgelist_names[:5])

[('"Acanthus spinosus"', '"Acmaeodera bipunctata "'), ('"Acanthus spinosus"', '"Adia cinerella "'), ('"Acanthus spinosus"', '"Alophora pusilla"'), ('"Acanthus spinosus"', '"Amasis lateralis"'), ('"Acanthus spinosus"', '"Amasis similis"')]


Now we can use the edgelist to create the bipartite graph. In two ways:

In [4]:
myGraph = BipartiteGraph(edgelist=edgelist_names)

In [5]:
myGraph = BipartiteGraph()
myGraph.set_edgelist(edgelist_names)

And now let's simply compute the bicm! This should run instantly. The solver checks that the solution is correct automatically.

In [6]:
myGraph.solve_tool()
dict_x, dict_y = myGraph.get_bicm_fitnesses()
print('Yielded data type is:', type(dict_x))

/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/numba/core/utils.py:612: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",
/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/bicm/models_functions.py:30: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  step_fun = args[0]
/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/bicm/models_functions.py:31: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  arg_step_fun = args[1]


max rows error = 7.144598246355827e-10
max columns error = 2.1845778519491432e-09
total error = 4.569030931378393e-09
Solver converged.
Yielded data type is: <class 'dict'>


/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/numba/core/utils.py:612: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",


The computed fitnesses are given as dictionaries, keeping track of the name of the nodes. If you are sure about the order of the fitnesses, you can get separately the fitness vectors and the dictionaries that keep track of the order of the nodes:

In [7]:
x = myGraph.x
y = myGraph.y
rows_dict = myGraph.rows_dict
cols_dict = myGraph.cols_dict

Alternatively, you can have the bicm matrix of probabilities, whose entries i, j are the bicm probabilities of having a link between nodes i and j (who will correspond to rows_dict[i] and cols_dict[j]), or compute it on your own

In [8]:
avg_mat = myGraph.get_bicm_matrix()
print(avg_mat[0, 0] == x[0] * y[0] / (1 + x[0] * y[0]))

True


In [9]:
avg_mat = bicm_from_fitnesses(x, y)
print(avg_mat[0, 0] == x[0] * y[0] / (1 + x[0] * y[0]))

True


And, if you like, you can check the solution by yourself:

In [10]:
# The data type conversions are generally consistent with the order, so this works
myGraph.check_sol(biad_mat, avg_mat)

max rows error = 7.144596025909777e-10
max columns error = 2.1846062736585736e-09
total error = 9.57038759352713e-09


You can then sample from the BiCM ensemble in two ways, via matrix sampling or edgelist sampling.

In [11]:
# Here we generate a sampling of 1000 networks and calculate the distribution of the number of links.
tot_links = []
for sample_i in range(1000):
    sampled_network = sample_bicm(avg_mat)
    tot_links.append(np.sum(sampled_network))

In [12]:
sampled_edgelist = sample_bicm_edgelist(x, y)

Now let's compute the projection: it can take some seconds.

In [13]:
rows_projection = myGraph.get_rows_projection()
print(rows_projection)

100%|████████████████████████████████████████| 131/131 [00:00<00:00, 588.51it/s]

{'"Fritillaria graeca"': ['"Heliotropium dolosum"', '"Hypochoeris achyrophorus"'], '"Echinops microcephalus"': ['"Fumana thymifolia"'], '"Centaurea raphanina"': ['"Echinops microcephalus"', '"Fumana thymifolia"'], '"Astragalus monspessulanus"': ['"Biscutella didyma"'], '"Fumana arabica"': ['"Papaver rhoeas"'], '"Asphodelus aestivus"': ['"Astragalus monspessulanus"', '"Biscutella didyma"'], '"Carduus pycnocephalus"': ['"Hirschfeldia incana"'], '"Phlomis fructicosa"': ['"Plantago lagopus"', '"Prasium majus"'], '"Heliotropium dolosum"': ['"Hypochoeris achyrophorus"'], '"Plantago lagopus"': ['"Prasium majus"']}


If you don't want to get the progress_bar you can set progress_bar=False. If you want to re-compute the projection with different settings (here a lower validation threshold), use compute_projection()

In [14]:
myGraph.compute_projection(rows=True, alpha=0.01, progress_bar=False)

In [15]:
# You can ask for an edgelist instead of an adjacency list by setting fmt="edgelist"
print('You can ask for an edgelist instead of an adjacency list by setting fmt="edgelist"')

rows_projection = myGraph.get_rows_projection(fmt='edgelist')
print(rows_projection)

You can ask for an edgelist instead of an adjacency list by setting fmt="edgelist"
[['"Echinops microcephalus"' '"Fumana thymifolia"']
 ['"Centaurea raphanina"' '"Echinops microcephalus"']
 ['"Centaurea raphanina"' '"Fumana thymifolia"']
 ['"Astragalus monspessulanus"' '"Biscutella didyma"']
 ['"Fumana arabica"' '"Papaver rhoeas"']
 ['"Asphodelus aestivus"' '"Astragalus monspessulanus"']
 ['"Asphodelus aestivus"' '"Biscutella didyma"']
 ['"Phlomis fructicosa"' '"Plantago lagopus"']
 ['"Phlomis fructicosa"' '"Prasium majus"']
 ['"Plantago lagopus"' '"Prasium majus"']]


These projection only contain links between nodes that behave similarly with respect to the expected behavior calculated from the BiCM. They could also be empty:

In [16]:
cols_projection = myGraph.get_cols_projection()

100%|████████████████████████████████████████| 666/666 [00:00<00:00, 817.56it/s]


No V-motifs will be validated. Try increasing alpha


In [17]:
# Increasing alpha too much will make the projection too dense. 
# It is not suggested: if it is empty with a high alpha, try different projections.
# Check the documentation and the papers for more info.
myGraph.compute_projection(rows=False, alpha=0.2)

100%|████████████████████████████████████████| 666/666 [00:01<00:00, 449.39it/s]

No V-motifs will be validated. Try increasing alpha


Different projection methods are implemented. 
They differ in the way of approximating the poisson-binomial distribution of the similarities between nodes.
By default, the poisson approximation is used, since the poisson binomial is very expensive to compute. Check the docs for more!
Here, an example of the poibin implementation. This can take even some minutes, and it's faster when using a biadjacency matrix instead of other data types. Note that the poisson approximation is fairly good.

In [18]:
myGraph.compute_projection(rows=False, method='poibin')
rows_projection = myGraph.get_rows_projection()
print(rows_projection)

Calculating p-values...


100%|████████████████████████████████████████| 465/465 [00:01<00:00, 330.16it/s]


No V-motifs will be validated. Try increasing alpha
{'"Echinops microcephalus"': ['"Fumana thymifolia"'], '"Centaurea raphanina"': ['"Echinops microcephalus"', '"Fumana thymifolia"'], '"Astragalus monspessulanus"': ['"Biscutella didyma"'], '"Fumana arabica"': ['"Papaver rhoeas"'], '"Asphodelus aestivus"': ['"Astragalus monspessulanus"', '"Biscutella didyma"'], '"Phlomis fructicosa"': ['"Plantago lagopus"', '"Prasium majus"'], '"Plantago lagopus"': ['"Prasium majus"']}


Now, let's have fun with other data structures and solver methods.

In [19]:
adj_list_names = adjacency_list_from_edgelist_bipartite(edgelist_names, convert_type=False)[0]

In [20]:
myGraph2 = BipartiteGraph(adjacency_list=adj_list_names)

Asking for the projection immediately will do everything automatically.

In [21]:
myGraph2.get_rows_projection()

First I have to compute the BiCM. Computing...
max rows error = 7.144598246355827e-10
max columns error = 2.1845778519491432e-09
total error = 4.569030931378393e-09
Solver converged.


/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/numba/core/utils.py:612: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",
100%|████████████████████████████████████████| 131/131 [00:00<00:00, 826.68it/s]


{'"Asphodelus aestivus"': ['"Astragalus monspessulanus"',
  '"Biscutella didyma"'],
 '"Astragalus monspessulanus"': ['"Biscutella didyma"'],
 '"Carduus pycnocephalus"': ['"Hirschfeldia incana"'],
 '"Centaurea raphanina"': ['"Echinops microcephalus"', '"Fumana thymifolia"'],
 '"Echinops microcephalus"': ['"Fumana thymifolia"'],
 '"Fritillaria graeca"': ['"Heliotropium dolosum"',
  '"Hypochoeris achyrophorus"'],
 '"Fumana arabica"': ['"Papaver rhoeas"'],
 '"Heliotropium dolosum"': ['"Hypochoeris achyrophorus"'],
 '"Phlomis fructicosa"': ['"Plantago lagopus"', '"Prasium majus"'],
 '"Plantago lagopus"': ['"Prasium majus"']}

In [22]:
rows_deg = biad_mat.sum(1)
cols_deg = biad_mat.sum(0)

In [23]:
myGraph3 = BipartiteGraph(degree_sequences=(rows_deg, cols_deg))

Default (recommended) method is 'newton', three more methods are implemented

In [24]:
myGraph3.solve_tool(method='fixed-point', tol=1e-5, exp=True)

max rows error = 7.97194843471516e-09
max columns error = 2.449649088021033e-07
total error = 3.4629640688343954e-07
Solver converged.


In [25]:
myGraph3.solve_tool(method='quasinewton', tol=1e-12, exp=True)

/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/numba/core/utils.py:612: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",
/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/bicm/models_functions.py:114: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  step_fun = args[0]
/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/bicm/models_functions.py:115: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  arg_step_fun = args[1]


max rows error = 5.133965288450781e-07
max columns error = 4.904200920918811e-07
total error = 4.534840823744446e-06
Solver converged.


/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/numba/core/utils.py:612: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",


This is the only method that works with a scipy solver, not recommended.

In [26]:
myGraph3.solve_tool(method='root')

max rows error = 2.1316282072803006e-14
max columns error = 1.4210854715202004e-14
total error = 2.715605518233133e-13
Solver converged.


If you want to retrieve the p-values of the projection on one layer, just call the pvals on the object after the computation:

In [31]:
# Use the rows arg to project on the rows or columns
myGraph2.compute_projection()
pval_adj_list = myGraph2.rows_pvals

100%|████████████████████████████████████████| 131/131 [00:00<00:00, 497.28it/s]


Keep in mind that the p-values are an adjacency list as well.

In [32]:
print(pval_adj_list[list(pval_adj_list.keys())[0]])

{1: 0.9972317357273726, 2: 0.9999999782267512, 3: 0.9923305795538132, 4: 0.9691310327757711, 5: 0.9958459965121682, 6: 0.9986966881488136, 7: 0.4201122733126731, 8: 0.9826101765475903, 9: 0.9980202118225047, 10: 0.999999924806693, 11: 0.9999930005461447, 12: 0.15580763169695316, 13: 0.09605763033353505, 14: 0.12026450868876538, 15: 0.9999980970151539, 16: 0.20607874995381165, 18: 0.12288836592288764, 19: 0.9045078785200347, 20: 0.5691142982511165, 21: 0.002227327838899109, 23: 0.8358035335155302, 24: 0.985081468592181, 25: 0.9996179582375733, 26: 0.999492110918237, 27: 0.7326793755863071, 28: 0.488107408748521, 29: 0.488107408748521, 30: 0.009012789568057232, 31: 0.5193898283428944, 33: 0.6305768333408953, 34: 0.19243479311418182, 35: 0.7011043550814545, 36: 0.9945819902758493, 37: 0.5921716747807659, 38: 0.1757461820681013, 39: 0.9998597991345853, 41: 0.9951651602560272, 42: 0.7699664957209895, 44: 0.9978043607403938, 45: 0.9994580504752817, 46: 0.9689984889420985, 47: 0.8073771209800